In [13]:
#installing dependencies
!pip install llama-index
!pip install llama-index-llms-google-genai
!pip install llama-index-embeddings-google-genai
!pip install google-generativeai
!pip install docx2txt

In [ ]:
#importing libraries
import os
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.google_genai import GoogleGenAI
import google.generativeai as genai
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.google_genai import GoogleGenAI

In [3]:
#importing api key from .env file
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure


<function google.generativeai.client.configure(*, api_key: 'str | None' = None, credentials: 'ga_credentials.Credentials | dict | None' = None, transport: 'str | None' = None, client_options: 'client_options_lib.ClientOptions | dict | None' = None, client_info: 'gapic_v1.client_info.ClientInfo | None' = None, default_metadata: 'Sequence[tuple[str, str]]' = ())>

In [5]:
#creating a document file from a wikipidea article
!pip install wikipedia python-docx


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11785 sha256=69d8f0b5a2163847cbeb414bbcb2e6005326d7ef0ab9db6e5a98d511ce0b0206
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\79\1d\c8\b64e19423cc5a2a339450ea5d145e7c8eb3d4aa2b150cde33b
Successfully built wikipedia

   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   ---------------------------------------- 0/2 [python-docx]
   -------------------- ------------------- 1/2 [wikipedia]
   ---------------------------------------- 2/2 [wikipedia]



  DEPRECATION: Building 'wikipedia' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wikipedia'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [9]:
#creating a document file from a wikipidea article

import wikipedia
from docx import Document
from urllib.parse import urlparse, unquote

def wikipedia_url_to_docx(url, filename="artificial_intelligence.docx", lang="en"):
    # Extract article title from URL
    parsed_url = urlparse(url)
    title = unquote(parsed_url.path.split("/")[-1].replace("_", " "))
    
    # Set language for Wikipedia API
    wikipedia.set_lang(lang)
    
    try:
        # Fetch Wikipedia page content
        page = wikipedia.page(title)
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation Error: Multiple pages found for '{title}'. Options include: {e.options}")
        return
    except wikipedia.exceptions.PageError:
        print(f"Page not found: '{title}'")
        return
    
    # Create and save docx file
    doc = Document()
    doc.add_heading(page.title, 0)
    
    for para in page.content.split('\n'):
        if para.strip():
            doc.add_paragraph(para.strip())
    
    doc.save(filename)
    print(f"Article '{page.title}' saved to {filename}")


wikipedia_url_to_docx("https://en.wikipedia.org/wiki/Artificial_intelligence", "Artificial intellience.docx")


Article 'Artificial intelligence' saved to Artificial intellience.docx


In [14]:
#loading the document file
documents = SimpleDirectoryReader(r'C:\Users\USER\rag_basics_1',  required_exts=[".docx"] ).load_data()

In [15]:
documents[0].text[:500]  # Display the first 500 characters of the first document

'Artificial intelligence\n\nArtificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.\n\nHigh-profi'

In [18]:
#declearing model and embedding and sentence splitter
llm = GoogleGenAI(model="gemini-2.5-pro", api_key  = api_key)
embed_model = GoogleGenAIEmbedding(model="text-embedding-004",  api_key  = api_key)
splitter = SentenceSplitter(chunk_size=512, chunk_overlap=50)


2025-09-09 11:11:13,662 - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro "HTTP/1.1 200 OK"


In [23]:
#indexing the document

Settings.node_parser = splitter  
Settings.llm = llm  
Settings.embed_model = embed_model
index = VectorStoreIndex.from_documents(documents)

2025-09-09 11:19:13,717 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-09-09 11:19:14,530 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-09-09 11:19:15,274 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-09-09 11:19:15,719 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-09-09 11:19:16,171 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"


In [ ]:
#storing the index on disk
index.storage_context.persist(persist_dir="./storage")

In [25]:
#querying the index
query_engine = index.as_query_engine()
response = query_engine.query("What is Artificial Intelligence?")

2025-09-09 11:20:25,719 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-09-09 11:20:25,805 - INFO - AFC is enabled with max remote calls: 10.
2025-09-09 11:20:40,137 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent "HTTP/1.1 200 OK"
2025-09-09 11:20:40,146 - INFO - AFC remote call 1 is done.


In [26]:
print(response)

Artificial intelligence (AI) is the ability of computational systems to carry out tasks that are typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. As a field of computer science research, it involves developing methods and software that allow machines to perceive their surroundings and take actions to maximize their chances of reaching specific goals.

There are several ways to define intelligence in this context:
*   **John McCarthy**, an AI founder, defined it as "the computational part of the ability to achieve goals in the world."
*   **Marvin Minsky**, another founder, described it as "the ability to solve hard problems."
*   **Google** defines it as the ability of systems to synthesize information, which is similar to how biological intelligence is defined.
*   A leading AI textbook describes the field as the study of agents that can perceive their environment and act to maximize their chances of achieving